In [1]:
import os

import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np

gpu_on = True

if gpu_on :
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

    
# K.set_floatx('float64')

from tensorflow.keras import mixed_precision 
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print(gpu_devices)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3070, compute capability 8.6
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from gan_module.data_loader.classification import ClassifyDataloader as ClassifyDataloader
from tensorflow.keras.applications.inception_v3 import preprocess_input
from glob import glob

data_set_name = "detect_lvi"
target_size = (512,512)
interpolation = "bilinear"
batch_size = 2
dtype="float32"

train_image_path_list = glob(f"./datasets/{data_set_name}/train/*/*")
valid_image_path_list = glob(f"./datasets/{data_set_name}/valid/*/*")
label_list = os.listdir(f"./datasets/{data_set_name}/train")
label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}

train_data_loader = ClassifyDataloader(image_path_list=train_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,
                                       shuffle=False,
                                       dtype=dtype
)
valid_data_loader = ClassifyDataloader(image_path_list=valid_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       dtype=dtype
)

In [3]:
from tensorflow.keras.applications import InceptionV3, ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.activations import sigmoid
import numpy as np
np.random.seed(1337)  # for reproducibility

DROPOUT_RATIO = 0.5

transfer_learning = False
train_mode = "include_depp_layer"

# create the base pre-trained model
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=train_data_loader.source_data_shape,
    classes=None,
    pooling=None,
    classifier_activation=None
)

if transfer_learning:
    if train_mode == "dense_only":
        base_model.trainable = False
    elif train_mode == "include_depp_layer":
        for layer in base_model.layers: 
            layer.trainable = False
            if layer.name == "mixed4":
                break

# add a global spatial average pooling layer
x = base_model.output
# (Batch_Size,?)
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT_RATIO)(x)
# let's add a fully-connected layer
# (Batch_Size,1)
x = Dense(1024, activation='relu')(x)
# (Batch_Size,1024)
x = Dropout(DROPOUT_RATIO)(x)
############## MODIFIED!!!!!!! 1 => 2 ##################
predictions = Dense(2, activation='softmax', dtype="float32")(x)


# this is the model we will train
model = Model(base_model.input, predictions)

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import Nadam

loss_function = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01)
optimizer = Nadam(1e-3, clipnorm=1)

# save_c = ModelCheckpoint(
#     "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5",
#     monitor='val_loss',
#     verbose=0,
#     save_best_only=False,
#     save_weights_only=True,
#     mode='min')

save_c = ModelCheckpoint(
    "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5",
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


reduceLROnPlat = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-9)
csv_logger = CSVLogger('./U_net/log.csv', append=False, separator=',')

def make_model_trainable(epoch, target_epoch):
    if epoch > target_epoch:
        model.trainable = True
        
make_trainable_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: make_model_trainable(epoch, target_epoch=10)
)


model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
start_epoch = 0
epochs = 200

model.fit(
    train_data_loader,
    validation_data=valid_data_loader,
    epochs=epochs,
    callbacks=[reduceLROnPlat, save_c, csv_logger, make_trainable_callback],
    initial_epoch=start_epoch
)

Epoch 1/200
4304/4304 [==============================] - 801s 181ms/step - loss: 0.7996 - accuracy: 0.5685 - val_loss: 1.2982 - val_accuracy: 0.5585
Epoch 2/200
4304/4304 [==============================] - 779s 181ms/step - loss: 0.6922 - accuracy: 0.6411 - val_loss: 9.4689 - val_accuracy: 0.5572
Epoch 3/200
 544/4304 [==>...........................] - ETA: 10:47 - loss: 0.6386 - accuracy: 0.7215